In [1]:
import psycopg2
import psycopg
from psycopg2 import OperationalError, InterfaceError
from google import genai
from google.genai import types
from google.genai.types import Part, UserContent
from utils import *
import logging 
from log_config import configure_logging
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
GENAI_API_KEY = os.getenv('GENAI_API_KEY')

In [2]:
sentiments_gemini = []
with open("sentiments_gemini.txt", 'r') as file:
    for line in file:
        sentiments_gemini.append(eval(line.strip()))

In [ ]:
HOST = 'database-sentiopen-aws-rds.c4j2kagc0s5z.us-east-1.rds.amazonaws.com'
PORT = 5432 
DBNAME = 'tensorflow_rds'
USER = 'postgres'
PASSWORD = ''

In [4]:
conn = None
cursor  = None

try:
    conn = psycopg2.connect(
        dbname = DBNAME,
        user = USER,
        password = PASSWORD,
        host = HOST,
        port = PORT
    )
    cursor = conn.cursor()
    print("Connected to the Database")
    save_sentiments_gemini(sentiments_gemini, cursor)
    conn.commit()
except Exception as e:
    print(f"Error occurred: {e}")

finally:
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

Connected to the Database


2025-05-14 10:49:29,222 - INFO - Trying to save sentiment (1/542)...
2025-05-14 10:49:29,369 - INFO - saved!
2025-05-14 10:49:29,369 - INFO - Trying to save sentiment (2/542)...
2025-05-14 10:49:29,516 - INFO - saved!
2025-05-14 10:49:29,517 - INFO - Trying to save sentiment (3/542)...
2025-05-14 10:49:29,664 - INFO - saved!
2025-05-14 10:49:29,665 - INFO - Trying to save sentiment (4/542)...
2025-05-14 10:49:29,812 - INFO - saved!
2025-05-14 10:49:29,813 - INFO - Trying to save sentiment (5/542)...
2025-05-14 10:49:29,963 - INFO - saved!
2025-05-14 10:49:29,964 - INFO - Trying to save sentiment (6/542)...
2025-05-14 10:49:30,117 - INFO - saved!
2025-05-14 10:49:30,117 - INFO - Trying to save sentiment (7/542)...
2025-05-14 10:49:30,264 - INFO - saved!
2025-05-14 10:49:30,265 - INFO - Trying to save sentiment (8/542)...
2025-05-14 10:49:30,412 - INFO - saved!
2025-05-14 10:49:30,413 - INFO - Trying to save sentiment (9/542)...
2025-05-14 10:49:30,573 - INFO - saved!
2025-05-14 10:49:30

In [ ]:
save_sentiments_gemini(sentiments_gemini, cursor)

In [ ]:
conn.commit()

In [ ]:
MODEL_GENAI = "gemini-2.0-flash" 

configure_logging()

In [ ]:
cursor.execute(
    """
    SELECT 
        issue_id,
        title,
        body
    FROM   
        issues_from_release
    WHERE
        release_number LIKE '%2.17%'
    """
)
issues = cursor.fetchall()
len(issues)

In [ ]:
cursor.execute(
    """
    SELECT 
        issue_id,
        title,
        body
    FROM   
        issues_from_release
    WHERE
        issue_id = 1882430710;
    """
)
issues = cursor.fetchall()
len(issues)

In [ ]:
client = genai.Client(api_key=GENAI_API_KEY)

In [ ]:
def classify_comment_sentiment_genai(comment_body, chat_session):
    prompt = f"Classify the sentiment of this comment: {comment_body}"

    response = chat_session.send_message(prompt)
    
    comment_sentiment = response.text.lower()

    logging.info(f"Sentiment of the comment is: {comment_sentiment}")

    return comment_sentiment


In [ ]:
def analyze_issue_sentiment_genai(issue_title, issue_body, comments, client, model):
    issue_chat_session = client.chats.create(
        model=model,
        history=[
            UserContent(parts=[Part(text=f"The title of the issue is: {issue_title}")]),
            UserContent(parts=[Part(text=f"The body of the issue is: {issue_body}")])
        ],
        config=types.GenerateContentConfig(
            temperature=0.2,
            system_instruction=(
                "You are a GitHub issue sentiment analysis expert. "
                "Classify each comment's sentiment as 'positive', 'negative', or 'neutral'. "
                "Answer with one word only."
            )
        )
    )

    results = []
    i = 1
    for comment in comments:
        comment_id, _, comment_body = comment

        logging.info(f"Classifying comment {comment_id} ({i}/{len(comments)})...")

        comment_sentiment = classify_comment_sentiment_genai(comment_body, issue_chat_session)

        results.append((comment_id, comment_sentiment))

        i += 1
    return results

In [ ]:
for issue in issues:
    issue_id, issue_title, issue_body = issue

    # Getting the data
    try:
        comments = get_comments_by_issue_id(issue_id, cursor)
        logging.info(f"{len(comments)} comments found for issue {issue_id}.")
        cursor.close()
        conn.close()
    except Exception as e:
        logging.error(f"Error when getting the comments for issue {issue_id}: {e}")
        continue

    # Analyzing sentiments
    try:
        logging.info(f"Starting sentiment analysis with Gemini 2.0 Flash for issue {issue_id}")
        sentiments_gemini = analyze_issue_sentiment_genai(
            issue_title=issue_title,
            issue_body=issue_body,
            comments=comments,
            client=client,
            model=MODEL_GENAI
        )
        logging.info(f"Sentiment analysis with Gemini 2.0 Flash concluded for issue {issue_id}.")
    except Exception as e:
        logging.error(f"Error when analyzing issue {issue_id} with Gemini 2.0 Flash.")
        continue

"""
    # Saving the data
    try:
        save_sentiments_gemini(sentiments_gemini, cursor)
        conn.commit()
        logging.info(f"Data successfully saved for issue {issue_id}.")
    except Exception as e:
        logging.error(f"Error when saving the data for issue {issue_id}.")
        continue
    
cursor.close()  
conn.close()
""" 

In [ ]:
type(sentiments_gemini)

In [ ]:
with open("sentiments_gemini.txt", 'w') as file:
    for item in sentiments_gemini:
        file.write(str(item) + '\n')